In [1]:
#!pip install parsivar
#!pip install libwapiti
!pip install hazm
!pip install seqeval
#!pip install spacy_huggingface_hub

  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=b472ac6c66f253c0ce065fbe0b395b11f2f42830912f7fd27c377939ca420fcd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)


cuda


In [ ]:
from parsivar import Tokenizer,Normalizer,POSTagger
my_normalizer = Normalizer()
my_tokenizer = Tokenizer()
my_tagger = POSTagger(tagging_model="wapiti")  # tagging_model = "wapiti" or "stanford". "wapiti" is faster than "stanford"
text_tags = my_tagger.parse(my_tokenizer.tokenize_words("مشهد به واسطه وجود حرم مطهر امام رضا ع و تولیتش، پتانسیل کمنظیری دارد که بعد از تهران این شهر را به دومین شهر مستعد در شب"))
print(text_tags)

[('مشهد', 'N_SING'), ('به', 'P'), ('واسطه', 'N_SING'), ('وجود', 'N_SING'), ('حرم', 'N_SING'), ('مطهر', 'ADJ'), ('امام', 'N_SING'), ('رضا', 'N_SING'), ('ع', 'N_SING'), ('و', 'CON'), ('تولیتش،', 'N_SING'), ('پتانسیل', 'N_SING'), ('کمنظیری', 'N_SING'), ('دارد', 'V_PRS'), ('که', 'CON'), ('بعد', 'ADV_TIME'), ('از', 'P'), ('تهران', 'N_SING'), ('این', 'DET'), ('شهر', 'N_SING'), ('را', 'CLITIC'), ('به', 'P'), ('دومین', 'ADJ_SUP'), ('شهر', 'N_SING'), ('مستعد', 'ADJ'), ('در', 'P'), ('شب', 'N_SING')]


In [ ]:
from hazm import POSTagger,word_tokenize
tagger = POSTagger(model='/content/drive/MyDrive/Kasre_Ez/pos_tagger.model')
text_tags = tagger.tag(word_tokenize('مشهد به واسطه وجود حرم مطهر امام رضا ع و تولیتش، پتانسیل کمنظیری دارد که بعد از تهران این شهر را به دومین شهر مستعد در شب.'))
print(text_tags)

[('مشهد', 'NOUN'), ('به', 'ADP'), ('واسطه', 'NOUN,EZ'), ('وجود', 'NOUN,EZ'), ('حرم', 'NOUN,EZ'), ('مطهر', 'ADJ,EZ'), ('امام', 'NOUN'), ('رضا', 'NOUN'), ('ع', 'NOUN'), ('و', 'CCONJ'), ('تولیتش', 'NOUN'), ('،', 'PUNCT'), ('پتانسیل', 'NOUN,EZ'), ('کمنظیری', 'NOUN'), ('دارد', 'VERB'), ('که', 'SCONJ'), ('بعد', 'ADP'), ('از', 'ADP'), ('تهران', 'NOUN'), ('این', 'DET'), ('شهر', 'NOUN'), ('را', 'ADP'), ('به', 'ADP'), ('دومین', 'NUM'), ('شهر', 'NOUN,EZ'), ('مستعد', 'ADJ'), ('در', 'ADP'), ('شب', 'NOUN'), ('.', 'PUNCT')]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

In [ ]:
import pandas as pd
from transformers import BertTokenizer

df = pd.read_csv('/content/lscp-50000.csv')

def create_labels(text, selected_words):
    if not isinstance(text, str):
        text = ''  # Convert non-string text to an empty string

    if not isinstance(selected_words, str):
        selected_words = ''

    selected_word_list = selected_words.split('،')

    tokenized_input = text.split()

    # Initialize labels as 'O' for every token
    labels = ['O'] * len(tokenized_input)

    for word in selected_word_list:
        word_tokens = word.split()
        n = len(word_tokens)
        for i in range(len(tokenized_input) - n + 1):
            if tokenized_input[i:i+n] == word_tokens:
                try:
                    labels[i] = 'B-WORD'
                except:
                  print('word_tokens',word_tokens)

                  print('tokenized_input',text)

                for j in range(1, n):
                    labels[i+j] = 'I-WORD'
                break

    return labels
output_data = []

for index, row in df.iterrows():
    sentence = row['text_query']
    selected = row['selected_words']

    labels = create_labels(sentence, selected)

    labels_str = ','.join(labels)

    output_data.append({'sentence': sentence, 'word_labels': labels_str})

output_df = pd.DataFrame(output_data)
print(output_df)

49996
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
word_tokens []
tokenized_input 
                                                sentence  \
0      بنر تصاویر دیکتاتور توسط کانونهایشورشی به آتش ...   
1      مشهد به واسطه وجود حرم مطهر امام رضاع و تولیتش...   
2      عاخه تو مغزشون کردن هرکسی که مثلا از نظام انتق...   
3                                         نرم باشه لطفا.   
4                          ولی پولشو می‌داد راضی‌تر بودم   
...                                                  ...   
49991   این حد از بیشعوری رو با عکس ائمه نشون ندین داعشی   
49992                                              به گا   
49993 

In [ ]:
data = output_df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

print(data.iloc[41].sentence)
print(data.iloc[41].word_labels)

انگلیس خواستار تحریم‌های اتحادیه اروپا علیه مادورو شد
O,B-WORD,B-WORD,B-WORD,O,B-WORD,O,O


In [ ]:
label2id={'I-WORD':2,'B-WORD': 1,'O': 0}
id2label={2:'I-WORD',1:'B-WORD',0:'O'}
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')


def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    if isinstance(sentence, float):
        sentence = str(sentence)  # Convert float to string

    sentence = sentence.strip()

    # Check if text_labels is empty or malformed
    if not text_labels or len(sentence.split()) != len(text_labels.split(",")):
        return None, None  # Indicate that this sample should be skipped

    for word, label in zip(sentence.split(), text_labels.split(",")):
        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]

        # Tokenization process
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # Ensure tokenized_sentence and labels are valid
        if tokenized_sentence is None or labels is None:
            print(f"Invalid tokenized sentence or labels at index {index}. Skipping entry.")
            return self.__getitem__((index + 1) % self.len)  # Skip and try the next index

        try:
            # Step 2: Add special tokens and corresponding labels
            tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"]
            labels = ["O"] + labels + ["O"]

            # Step 3: Truncate or pad the tokenized sentence and labels to max length
            maxlen = self.max_len
            if len(tokenized_sentence) > maxlen:
                tokenized_sentence = tokenized_sentence[:maxlen]
                labels = labels[:maxlen]
            else:
                tokenized_sentence += ['[PAD]'] * (maxlen - len(tokenized_sentence))
                labels += ["O"] * (maxlen - len(labels))

            # Step 4: Create attention mask
            attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

            # Step 5: Convert tokens to input IDs
            ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

            # Step 6: Convert labels to IDs
            label_ids = [label2id[label] for label in labels]

        except Exception as e:
            print(f"Error processing index {index}: {e}")
            return self.__getitem__((index + 1) % self.len)  # Skip the current entry

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(attn_mask, dtype=torch.long),
            'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
train_size = 0.8
MAX_LEN = 128
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (38659, 2)
TRAIN Dataset: (30927, 2)
TEST Dataset: (7732, 2)


In [ ]:
testing_set[0]

{'ids': tensor([    2,  5574, 12139, 14744,  1012,     4,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
کاری        O
به          O
این         O
ندارم       O
ولی         O
کسی         O
که          O
شمارش       B-WORD
##و         B-WORD
میذاره      B-WORD
پسو         B-WORD
##ورد       B-WORD
ساده        O
لوحه        O
.           O
؟           O
[SEP]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O


In [ ]:
#MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 16
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


In [ ]:
model = BertForTokenClassification.from_pretrained('HooshvareLab/bert-fa-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)

model.to(device)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [ ]:

ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.1718, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 3])

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        active_accuracy = mask.view(-1) == 1 # shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.1768450736999512
Training loss per 100 training steps: 0.13366310731830575
Training loss per 100 training steps: 0.09513311094571998
Training loss per 100 training steps: 0.07947372157819742
Training loss per 100 training steps: 0.06943823001405544
Training loss per 100 training steps: 0.06317980769657328
Training loss per 100 training steps: 0.05894461352712848
Training loss per 100 training steps: 0.055862814438437644
Training loss per 100 training steps: 0.05302286856911249
Training loss per 100 training steps: 0.050898635550634674
Training loss per 100 training steps: 0.04904259838857925
Training loss per 100 training steps: 0.04749295397701383
Training loss per 100 training steps: 0.045997574451619096
Training loss per 100 training steps: 0.04472272224376389
Training loss per 100 training steps: 0.043468505432955565
Training loss per 100 training steps: 0.04238247493456312
Training loss per 100 training steps: 0.04148109102

In [ ]:
model.to(device)
def valid(model, testing_loader):
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            if batch is None:  # Skip invalid batches
                continue

            ids = batch['ids'].to(device, dtype=torch.long)
            mask = batch['mask'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()
            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100 == 0:
                print(f"Validation loss per 100 evaluation steps: {eval_loss / nb_eval_steps}")

            # Compute accuracy
            flattened_targets = targets.view(-1)
            active_logits = eval_logits.view(-1, model.num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)
            active_accuracy = mask.view(-1) == 1
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    eval_loss /= nb_eval_steps
    eval_accuracy /= nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return eval_labels, eval_preds
labels, predictions = valid(model, testing_loader)


Validation loss per 100 evaluation steps: 0.05145786702632904
Invalid tokenized sentence or labels at index 1245. Skipping entry.
Validation loss per 100 evaluation steps: 0.0193598467515468
Validation loss per 100 evaluation steps: 0.019549503674573122
Validation loss per 100 evaluation steps: 0.01935605811221259
Validation loss per 100 evaluation steps: 0.019305186031474512
Validation Loss: 0.01910663925689522
Validation Accuracy: 0.9503086591693927


In [ ]:
for param in model.parameters(): param.data = param.data.contiguous()
model.save_pretrained('/content/drive/MyDrive/Kasre_Ez/models_5000')

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/Kasre_Ez/models_5000')

('/content/drive/MyDrive/Kasre_Ez/models_5000/tokenizer_config.json',
 '/content/drive/MyDrive/Kasre_Ez/models_5000/special_tokens_map.json',
 '/content/drive/MyDrive/Kasre_Ez/models_5000/vocab.txt',
 '/content/drive/MyDrive/Kasre_Ez/models_5000/added_tokens.json',
 '/content/drive/MyDrive/Kasre_Ez/models_5000/tokenizer.json')

In [ ]:

from transformers import AutoTokenizer, AutoModelForTokenClassification
model_name = "/content/drive/MyDrive/Kasre_Ez/models_folder"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


In [ ]:

from seqeval.metrics import classification_report
labels = [label.item() for label in labels]  # If labels are tensors
predictions = [prediction.item() for prediction in predictions]  # If predictions are tensors

labels = [id2label[label_id] for label_id in labels]  # Mapping label IDs to label names
predictions = [id2label[pred_id] for pred_id in predictions]  # Mapping prediction IDs to label names

print(classification_report([labels], [predictions]))


              precision    recall  f1-score   support

        WORD       0.84      0.87      0.85     26548

   micro avg       0.84      0.87      0.85     26548
   macro avg       0.84      0.87      0.85     26548
weighted avg       0.84      0.87      0.85     26548



###Fine_tune Again

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        active_accuracy = mask.view(-1) == 1 # shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.01758834533393383
Training loss per 100 training steps: 0.01977159732971156
Training loss epoch: 0.018722230637649773
Training accuracy epoch: 0.9563997475992332


In [ ]:
model.to(device)
def valid(model, testing_loader):
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            if batch is None:  # Skip invalid batches
                continue

            ids = batch['ids'].to(device, dtype=torch.long)
            mask = batch['mask'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()
            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100 == 0:
                print(f"Validation loss per 100 evaluation steps: {eval_loss / nb_eval_steps}")

            # Compute accuracy
            flattened_targets = targets.view(-1)
            active_logits = eval_logits.view(-1, model.num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)
            active_accuracy = mask.view(-1) == 1
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    eval_loss /= nb_eval_steps
    eval_accuracy /= nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return eval_labels, eval_preds
labels, predictions = valid(model, testing_loader)


Validation loss per 100 evaluation steps: 0.00766481040045619
Validation Loss: 0.015229172247927636
Validation Accuracy: 0.9601837151280009


In [ ]:
#results of finetuned twice model
from seqeval.metrics import classification_report
labels = [label.item() for label in labels]  # If labels are tensors
predictions = [prediction.item() for prediction in predictions]  # If predictions are tensors

labels = [id2label[label_id] for label_id in labels]  # Mapping label IDs to label names
predictions = [id2label[pred_id] for pred_id in predictions]  # Mapping prediction IDs to label names

print(classification_report([labels], [predictions]))


              precision    recall  f1-score   support

        WORD       0.87      0.82      0.84       337

   micro avg       0.87      0.82      0.84       337
   macro avg       0.87      0.82      0.84       337
weighted avg       0.87      0.82      0.84       337



In [ ]:
# Results of first model
from seqeval.metrics import classification_report
labels = [label.item() for label in labels]  # If labels are tensors
predictions = [prediction.item() for prediction in predictions]  # If predictions are tensors

labels = [id2label[label_id] for label_id in labels]  # Mapping label IDs to label names
predictions = [id2label[pred_id] for pred_id in predictions]  # Mapping prediction IDs to label names

print(classification_report([labels], [predictions]))


              precision    recall  f1-score   support

        WORD       0.72      0.90      0.80       337

   micro avg       0.72      0.90      0.80       337
   macro avg       0.72      0.90      0.80       337
weighted avg       0.72      0.90      0.80       337

